--------------




## Custom Dataset

`torch.utils.data.dataset`을 상속받아서 구현.

Dataset 설정 시 꼭 필요한 Methods:

1. `__init__`: 데이터셋 (입력, 출력 파일들) 경로 및 필요한 인스턴스(변수, 객체 등) 초기화
   
  - 일반적으로 필요한 인스턴스 (인스턴스 이름은 임의로 설정해도 무관)
  
    - `self.input_dir`: 입력 파일들이 들어있는 경로.
    - `self.label_dir`: 답안 파일들이 들어있는 경로. 만약 답안이 하나의 파일 안에 들어있는 경우 파일로 초기화 함.
    - `self.transform`: 입력 영상들을 불러와서 처리하기 위한 객체. 보통 `torchvision.transforms`의 클래스들로 구현. 없는 경우 None.
    - `self.target_transform`: 답안 파일들을 불러와서 처리하기 위한 객체. 없는 경우 None.

2. `__len__`: 데이터베이스내의 샘플 개수. 리스트의 길이를 출력하는 등의 방식으로 출력. `len(dataset)`을 사용할 수 있게 해줌.
   
3. `__getitem__`: 인덱스(몇 번째 파일을 불러올지) idx가 주어졌을 때, idx에 해당하는 입력과 답안을 출력해 주는 Method. idx는 dataloader에서 제공.


In [ ]:
# 예시
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

`__init__`에서 미리 파일 리스트를 받아놓는 방법. 또는 경로만 초기화 하고 `__getitem__`에서 불러오는 방법이 있음.

In [ ]:
# 예제 1. 입력, 답안 모두 파일일 때. __init__에서 경로만 저장.
def __init__(self, input_dir, label_dir, transform=None, target_transform=None):
    self.input_dir = input_dir
    self.label_dir = label_dir
    self.transform = transform
    self.target_transform = target_transform
    
def __len__(self):
    return len(os.listdir(input_dir))
    
def __getitem__(self, idx):
    input_filename = os.path.join(self.img_dir, 'input_' + str(idx) + '.jpg')   # 파일 이름이 input_0xx.jpg 라고 가정.
    label_filename = os.path.join(self.label_dir, 'label_' + str(idx) + '.jpg')   # 파일 이름이 label_0xx.jpg 라고 가정.
    
    input = read_image(input_filename)
    label = read_image(label_filename)

    if self.transform is not None:
        input = self.transform(input)
    
    if self.target_transform is not None:
        label = self.target_transform(label)
    
    return input, label

In [ ]:
# 예제 2. 입력, 답안 모두 파일일 때. __init__에서 파일 이름을 리스트로 미리 저장
def __init__(self, input_dir, label_dir, transform=None, target_transform=None):
    self.input_list = os.listdir(input_dir)
    self.label_list = os.listdir(label_dir)
    self.transform = transform
    self.target_transform = target_transform
    
def __len__(self):
    return len(self.input_list)
    
def __getitem__(self, idx):
    input = read_image(self.input_list[idx])
    label = read_image(self.label_list[idx])

    if self.transform is not None:
        input = self.transform(input)
    
    if self.target_transform is not None:
        label = self.target_transform(label)
    
    return input, label

`Dataset`에서 개별 파일을 불러오는 부분을 처리해 준 후 `torch.utils.Dataloader`에 입력해주면 dataloader에 설정된 batch size만큼 개별 파일들을 불러와서 tensor 형태로 묶어줌.

In [ ]:
dataset = CustomImageDataset(input_dir, label_dir, transform, target_transform)
dataloader = torch.utils.DataLoader(dataset, 
                                    batch_size=128, 
                                    shuffle=True)  # 미니배치 처리를 담당